In [ ]:
# Устанавливаем OpenJDK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Закачиваем Spark
!wget -q http://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz -O spark.tgz
# Распаковываем архив со Spark
!tar xf spark.tgz
# Устанавливаем пакет findspark для работы со Spark из Python
!pip install -q findspark
# Настраиваем переменные окружения для работы с Apache Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"
# Находим установку Spark
import findspark
findspark.init()
# Подключаем необходимые модули для работы со Spark из Python
from pyspark.sql import SparkSession
# Создаем сессию Spark на локальном компьютере
spark = SparkSession.builder.master("local[*]").getOrCreate()
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip -O /content/ml-100k.zip -q
!unzip -qq /content/ml-100k.zip -d "sample_data/"
!rm spark.tgz
!rm ml-100k.zip


In [ ]:
import collections
movieRDD = spark.sparkContext.textFile("/content/sample_data/ml-100k/u.data")

##Задача 1
* вычислять и выводить на экран статистику по числу оценок для каждого фильма
* вычислять и выводить на экран статистику по числу оценок для всех фильмов


In [ ]:
def printStat(inp):
  marks = [0 for i in range(0, 5)] # Create list of zeroes with length 5 (becouse max mark - 5 stars)
  RDD = movieRDD.map(lambda review: review.split('\t'))
     
  if inp != 'ALL': # if we want to check marks of specific movie
    RDD = RDD \
      .filter(lambda x: x[1] == str(inp)) \
      
  RDD = RDD.map(lambda review: int(review[2])) # select only marks
  for mark in RDD.collect():
    marks[mark-1]+=1 # increment mark counter
  print(f'Marks for film {inp}: 1 -> {marks[0]}, 2 -> {marks[1]}, 3 -> {marks[2]}, 4 -> {marks[3]}, 5 -> {marks[4]}') # print results

Проверка:

In [ ]:
RDD = movieRDD \
  .map(lambda review: review.split('\t')) \
  .map(lambda review: int(review[1])) \
  .distinct() \
  .sortBy(lambda x: x)

for i in RDD.take(10):
  printStat(i)

Marks for film 1: 1 -> 8, 2 -> 27, 3 -> 96, 4 -> 202, 5 -> 119
Marks for film 2: 1 -> 8, 2 -> 17, 3 -> 55, 4 -> 42, 5 -> 9
Marks for film 3: 1 -> 11, 2 -> 20, 3 -> 25, 4 -> 23, 5 -> 11
Marks for film 4: 1 -> 6, 2 -> 24, 3 -> 57, 4 -> 93, 5 -> 29
Marks for film 5: 1 -> 4, 2 -> 11, 3 -> 32, 4 -> 33, 5 -> 6
Marks for film 6: 1 -> 2, 2 -> 4, 3 -> 5, 4 -> 7, 5 -> 8
Marks for film 7: 1 -> 9, 2 -> 31, 3 -> 89, 4 -> 164, 5 -> 99
Marks for film 8: 1 -> 4, 2 -> 13, 3 -> 47, 4 -> 71, 5 -> 84
Marks for film 9: 1 -> 11, 2 -> 18, 3 -> 59, 4 -> 114, 5 -> 97
Marks for film 10: 1 -> 2, 2 -> 7, 3 -> 21, 4 -> 33, 5 -> 26


In [ ]:
printStat(346)
printStat(474)
printStat(265)
printStat(465)
printStat(451)
printStat(86)
printStat(257)
printStat(222)
printStat(40)
printStat(29)
printStat("ALL")

Marks for film 346: 1 -> 7, 2 -> 10, 3 -> 32, 4 -> 49, 5 -> 28
Marks for film 474: 1 -> 0, 2 -> 6, 3 -> 34, 4 -> 59, 5 -> 95
Marks for film 265: 1 -> 1, 2 -> 13, 3 -> 62, 4 -> 91, 5 -> 60
Marks for film 465: 1 -> 4, 2 -> 8, 3 -> 26, 4 -> 30, 5 -> 17
Marks for film 451: 1 -> 15, 2 -> 31, 3 -> 37, 4 -> 54, 5 -> 33
Marks for film 86: 1 -> 4, 2 -> 10, 3 -> 23, 4 -> 67, 5 -> 46
Marks for film 257: 1 -> 2, 2 -> 28, 3 -> 81, 4 -> 126, 5 -> 66
Marks for film 222: 1 -> 7, 2 -> 30, 3 -> 108, 4 -> 155, 5 -> 65
Marks for film 40: 1 -> 9, 2 -> 9, 3 -> 20, 4 -> 17, 5 -> 2
Marks for film 29: 1 -> 15, 2 -> 34, 3 -> 45, 4 -> 14, 5 -> 6
Marks for film ALL: 1 -> 6110, 2 -> 11370, 3 -> 27145, 4 -> 34174, 5 -> 21201


##Задача 2
* Очистить текст от знаков препинания и пустых строк
* Перевести в нижний регистр и разделить по пробелам
* Подсчитать наиболее часто встречающиеся символы
* Использовать RDD

In [48]:
!wget http://www.lib.ru/INOOLD/BALZAK/shagren.txt_Ascii.txt | iconv -f cp1251
import string
i = 0
with open('/content/shagren.txt_Ascii.txt', encoding="koi8_r") as inF, open('/content/shagren.txt_utf8.txt', "w") as outF: #read and write txt files
  for line in inF:
    line = line.translate(str.maketrans('', '', string.punctuation)) # remove punctuation chars
    outF.write(line)
!rm /content/shagren.txt_Ascii.txt

--2022-12-02 11:43:23--  http://www.lib.ru/INOOLD/BALZAK/shagren.txt_Ascii.txt
Resolving www.lib.ru (www.lib.ru)... 81.176.66.163
Connecting to www.lib.ru (www.lib.ru)|81.176.66.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘shagren.txt_Ascii.txt’

shagren.txt_Ascii.t     [      <=>           ] 510.68K   303KB/s    in 1.7s    

2022-12-02 11:43:27 (303 KB/s) - ‘shagren.txt_Ascii.txt’ saved [522937]



In [78]:
bookRDD = spark.sparkContext.textFile("/content/shagren.txt_utf8.txt")

In [79]:
bookRDD = bookRDD.flatMap(lambda word: word.split(' ')) \
  .filter(lambda word: word != '') \
  .map(lambda word: word.lower()) \
  .map(lambda word: [word, 1]) #create flatMap with split function, then filtrate by empty strings and create dictionaty for words in primary text
bookRDD.take(30)

[['\x14оноре', 1],
 ['де', 1],
 ['бальзак', 1],
 ['шагреневая', 1],
 ['кожа\x15', 1],
 ['перевод', 1],
 ['бгрифцов', 1],
 ['ocr', 1],
 ['владимир', 1],
 ['лущенко', 1],
 ['httpvladmrlchatrulibraryhtm', 1],
 ['origin', 1],
 ['httpluschenkovichatrubalzachtm', 1],
 ['господину', 1],
 ['савари', 1],
 ['члену', 1],
 ['академии', 1],
 ['наук', 1],
 ['стерн', 1],
 ['тристрам', 1],
 ['шенди', 1],
 ['гл', 1],
 ['cccxxii', 1],
 ['\x14i', 1],
 ['талисман\x15', 1],
 ['в', 1],
 ['конце', 1],
 ['октября', 1],
 ['1829', 1],
 ['года', 1]]

In [73]:
bookRDD=bookRDD \
  .reduceByKey(lambda a,b: a+b) \
  .sortBy(lambda word: word[1], ascending=False) # group by key and sort in decrease order

bookRDD.take(10)

[('и', 2204),
 ('в', 1977),
 ('я', 1252),
 ('не', 1247),
 ('на', 1094),
 ('он', 755),
 ('как', 717),
 ('с', 693),
 ('что', 653),
 ('его', 502)]